task name: Implement ProduceHashedNgrams
task description: Implement the provided C# code for producing hashed n-grams using ML.NET

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
public class TextData
{
    public string Text { get; set; }
}

var mlContext = new MLContext();
var samples = new List<TextData>()
{
    new TextData(){ Text = "This is an example to compute n-grams using hashing." },
    new TextData(){ Text = "N-gram is a sequence of 'N' consecutive words/tokens." },
    new TextData(){ Text = "ML.NET's ProduceHashedNgrams API produces count of n-grams and hashes it as an index into a vector of given bit length." },
    new TextData(){ Text = "The hashing reduces the size of the output feature vector" },
    new TextData(){ Text = "which is useful in case when number of n-grams is very large." }
};
var dataview = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var textPipeline = mlContext.Transforms.Text.TokenizeIntoWords("Tokens", "Text")
    .Append(mlContext.Transforms.Conversion.MapValueToKey("Tokens"))
    .Append(mlContext.Transforms.Text.ProduceHashedNgrams(
        "NgramFeatures", "Tokens",
        numberOfBits: 5,
        ngramLength: 3,
        useAllLengths: false,
        maximumNumberOfInverts: 1));

var textTransformer = textPipeline.Fit(dataview);
var transformedDataView = textTransformer.Transform(dataview);

In [ ]:
public class TransformedTextData : TextData
{
    public float[] NgramFeatures { get; set; }
}

var predictionEngine = mlContext.Model.CreatePredictionEngine<TextData, TransformedTextData>(textTransformer);
var prediction = predictionEngine.Predict(samples[0]);

In [ ]:
Console.WriteLine("Number of Features: " + prediction.NgramFeatures.Length);
//... more code to print n-grams and features